In [4]:
import gzip
import shutil

input_file = "/data/sr933/scRCC validation/GSE242299/GSE242299_all_cells_50236_33538.h5ad.gz"
output_file = "/data/sr933/scRCC validation/GSE242299/GSE242299_all_cells_50236_33538.h5ad"

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [6]:
import scanpy as sc

# Load the .h5ad file
adata = sc.read("/data/sr933/scRCC validation/GSE242299/GSE242299_all_cells_50236_33538.h5ad")


In [9]:
# Check the structure of the AnnData object
print(adata)

# Explore the expression matrix
# The expression matrix is typically stored in the .X attribute
print(adata.X.shape)  # Dimensions of the expression matrix (cells x genes)

# View the first few rows of the expression matrix (dense form)
print(adata.X[:5, :5])

# Metadata (obs) - information about cells (e.g., cell types, experimental conditions)
print(adata.total_counts.head())






AnnData object with n_obs × n_vars = 50236 × 33538
    obs: 'library', 'total_counts', 'pct_counts_mito', 'library2', 'sample', 'patient', 'pT stage', 'seq_date', 'beads', 'operation', 'sex', 'tumor size, mm', 'age', 'tissue', 'necrosis', 'doublet_score', 'n_counts', 'sp_cl_43', 'cell_type', 'broad_cell_type', 'color', 'cell_group', 'stage_color', 'patient_color', 'group_color', 'library_name'
    uns: 'X_lin_cptt', 'X_log_z', 'beads_colors', 'draw_graph', 'neighbors', 'pca', 'sample_colors', 'seq_date_colors', 'tissue_colors', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_pca_harmony', 'X_umap'
(50236, 33538)
<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 0 stored elements and shape (5, 5)>


AttributeError: 'AnnData' object has no attribute 'total_counts'

In [10]:
# Extract cell annotations
cell_annotations = adata.obs

# Save to a CSV file (optional)
cell_annotations.to_csv("cell_annotations.csv")

In [11]:
import pandas as pd

# Extract expression matrix as a dense DataFrame
expression_matrix = pd.DataFrame(
    adata.X.toarray(), 
    index=adata.obs_names, 
    columns=adata.var_names
)

# Save to a CSV file (optional)
expression_matrix.to_csv("expression_matrix.csv")


In [19]:
import pandas as pd

# Load the expression DataFrame
expression_df = pd.read_csv("/data/sr933/scRCC validation/processed data/expression_matrix.csv", index_col=0)  # Ensure index_col=0 if the first column contains cell IDs




KeyboardInterrupt: 

In [ ]:
# Load the target genes
target_genes_df = pd.read_csv("/home/sr933/rcc/data/Target_genes.csv")
target_genes_list = target_genes_df['Genes'].tolist()

# Filter the columns of the expression DataFrame to keep only target genes
filtered_expression_df = expression_df.loc[:, expression_df.columns.isin(target_genes_list)]



# Print a summary
print(f"Original expression matrix shape: {expression_df.shape}")
print(f"Filtered expression matrix shape: {filtered_expression_df.shape}")

In [18]:
# Identify the missing genes
missing_genes = [gene for gene in target_genes_list if gene not in filtered_expression_df.columns]

# Print the missing genes
if missing_genes:
    print(f"The following genes are missing from the filtered DataFrame: {missing_genes}")
else:
    print("All target genes are present in the filtered DataFrame.")

# Add a new column 'C10orf10' with all values set to 0
filtered_expression_df.loc[:, 'C10orf10'] = 0

# Sort the columns of final_df based on the order in target_genes_list
# Retain only the columns that are in target_genes_list
sorted_df = filtered_expression_df[target_genes_list]

All target genes are present in the filtered DataFrame.


In [20]:
print(sorted_df.shape)

(50236, 96)


In [21]:
annotation_df=pd.read_csv("/data/sr933/scRCC validation/processed data/cell_annotations.csv")
labels=annotation_df["cell_group"].to_numpy()
data=sorted_df.to_numpy().T

In [22]:
import pickle
validation_data={"X_test": data, "y_test":labels}
# Path to save the pickle file
pickle_file_path = "/data/sr933/scRCC validation/processed data/scRCC_validation_dataset_lithuanian.pkl"  # Update with your desired path

# Save the validation_data dictionary as a pickle file
with open(pickle_file_path, "wb") as pkl_file:
    pickle.dump(validation_data, pkl_file)